In [1]:
from sort import *
import numpy as np
import timeit



c:\ProgramData\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\ProgramData\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
sample_path = r"G:\My Drive\my_work\sensitive_objects_detection_HD\exe_window\det_logs\2024-03-08_15-24-53_____9.txt"

In [19]:
# read the sample file
with open(sample_path, "r") as file:
    data = file.readlines()
stride = 5

In [20]:
for d in data:
    print(d)

INFO:root:5;flag;615.4624633789062,131.50624084472656,657.8624877929688,197.42498779296875;0.78;848,480

INFO:root:5;flag;347.8124694824219,130.18124389648438,369.67498779296875,193.11874389648438;0.44;848,480

INFO:root:5;flag;344.83123779296875,129.2703094482422,373.9812316894531,257.04998779296875;0.42;848,480

INFO:root:10;flag;615.793701171875,132.1687469482422,657.5311889648438,198.41873168945312;0.78;848,480

INFO:root:10;flag;346.8187255859375,130.59530639648438,368.6812438964844,193.4499969482422;0.44;848,480

INFO:root:10;flag;345.1624755859375,130.59530639648438,374.3124694824219,257.7124938964844;0.44;848,480

INFO:root:15;flag;616.7874755859375,133.57655334472656,657.8624877929688,201.06874084472656;0.78;848,480

INFO:root:15;flag;344.1687316894531,131.83749389648438,374.64373779296875,260.6937255859375;0.48;848,480

INFO:root:15;flag;346.48748779296875,131.92030334472656,368.3499755859375,195.43748474121094;0.45;848,480

INFO:root:20;flag;617.449951171875,135.481246948242

In [21]:
data_dict = {}
width, heihgt = 0, 0
for line in data:
    line = line[10:]
    line = line.split(";")
    frame_number = int(line[0])
    width, heihgt = int(line[4]), int(line[5])
    class_name = line[1]
    bbox = [int(float(i)) for i in line[2].split(",")]
    score = float(line[3])
    detection = bbox + [score]
    detection = np.array(detection)
    detection = np.expand_dims(detection, axis=0)
    if frame_number not in list(data_dict.keys()):
        data_dict[frame_number] = detection
    else:
        data_dict[frame_number] = np.vstack((data_dict[frame_number], detection))

In [28]:
tracker = Sort(max_age=3, min_hits=1, iou_threshold=0.1)
tracker.reset() # set the counter to 0 every new video
id_counter_dict = {}
data_dict_with_id = {}
start_time = timeit.default_timer()
for i in range(0, max(list(data_dict.keys())), stride):
    if i not in list(data_dict.keys()):
        detection = np.array([[0, 0, 0, 0, 0]])
        state = tracker.update(detection)
        continue
    # check if data_dict[i] is tuple or not
    if type(data_dict[i]) == np.ndarray:
        try:
            detection = data_dict[i] #.reshape(1, -1)
            #start_time = timeit.default_timer()
            state = tracker.update(detection)
            #print("time: ", timeit.default_timer() - start_time)
        except:
            print("error")
            print(data_dict[i])
        #state = tracker.update(data_dict[i])
        for res in state:
            res = res.astype(int)
            x1, y1, x2, y2, id = res
            
            if id not in id_counter_dict:
                id_counter_dict[id] = []
            id_counter_dict[id].append(i)
            
            print("frame: ", i, "id: ", id, "x1: ", x1, "y1: ", y1, "x2: ", x2, "y2: ", y2)
            if i not in list(data_dict_with_id.keys()):
                data_dict_with_id[i] = np.array([x1, y1, x2, y2, id])
                data_dict_with_id[i] = np.expand_dims(data_dict_with_id[i], axis=0)
            else:
                data_dict_with_id[i] = np.vstack((data_dict_with_id[i], [x1, y1, x2, y2, id]))
    else:
        print(data_dict[i])
        print("error")

print("time: ", timeit.default_timer() - start_time)
    

e:\HD_VNese_map\main_src\exe_window\sort.py:77: RuntimeWarning: invalid value encountered in divide
  r = w / float(h)


frame:  10 id:  4 x1:  345 y1:  129 x2:  373 y2:  257
frame:  10 id:  3 x1:  346 y1:  130 x2:  368 y2:  193
frame:  10 id:  2 x1:  615 y1:  131 x2:  657 y2:  197
frame:  15 id:  4 x1:  344 y1:  130 x2:  373 y2:  260
frame:  15 id:  3 x1:  345 y1:  131 x2:  367 y2:  194
frame:  15 id:  2 x1:  615 y1:  133 x2:  657 y2:  200
frame:  20 id:  4 x1:  344 y1:  131 x2:  373 y2:  262
frame:  20 id:  3 x1:  346 y1:  131 x2:  368 y2:  195
frame:  20 id:  2 x1:  616 y1:  135 x2:  658 y2:  201
frame:  25 id:  5 x1:  566 y1:  136 x2:  594 y2:  206
frame:  25 id:  4 x1:  343 y1:  133 x2:  374 y2:  264
frame:  25 id:  3 x1:  347 y1:  133 x2:  369 y2:  196
frame:  25 id:  2 x1:  616 y1:  136 x2:  658 y2:  204
frame:  30 id:  5 x1:  566 y1:  136 x2:  593 y2:  208
frame:  30 id:  4 x1:  343 y1:  134 x2:  374 y2:  266
frame:  30 id:  3 x1:  347 y1:  135 x2:  369 y2:  198
frame:  30 id:  2 x1:  616 y1:  137 x2:  658 y2:  205
frame:  35 id:  5 x1:  566 y1:  135 x2:  594 y2:  207
frame:  35 id:  3 x1:  346 y

frame:  925 id:  135 x1:  397 y1:  231 x2:  434 y2:  285
frame:  925 id:  134 x1:  595 y1:  235 x2:  660 y2:  274
frame:  1120 id:  173 x1:  231 y1:  131 x2:  300 y2:  169
time:  0.24171350000005987


: 

In [24]:
videopath = r"C:\Users\Administrator\Videos\9.mp4"
import cv2
video = cv2.VideoCapture(videopath)
while True:
    ret, frame = video.read()
    if ret:
        frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_number in list(data_dict_with_id.keys()):
            bboxs = data_dict_with_id[frame_number][:, :4]
            ids = data_dict_with_id[frame_number][:, 4]
            for i, bbox in enumerate(bboxs):
                x1, y1, x2, y2 = bbox
                id = ids[i]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, str(id), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("frame", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        cv2.waitKey(20)
    else:
        break
video.release()
cv2.destroyAllWindows()


In [25]:
for i in list(id_counter_dict.keys()):
    print(i, (id_counter_dict[i]))

4 [10, 15, 20, 25, 30, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
3 [10, 15, 20, 25, 30, 35, 45]
2 [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110]
5 [25, 30, 35, 40, 45, 50, 55, 60, 65, 75, 80, 85, 90, 95, 100]
58 [380, 385]
59 [390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440]
60 [405, 420]
128 [785, 790, 800, 805, 810, 815, 820, 825]
135 [855, 860, 875, 880, 885, 890, 895, 900, 905, 910, 915, 920, 925]
134 [855, 860, 865, 870, 875, 880, 885, 890, 895, 900, 905, 910, 915, 920, 925]
173 [1120]


In [15]:
data_dict.keys()

dict_keys([5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 780, 785, 790, 800, 805, 810, 815, 820, 825, 850, 855, 860, 865, 870, 875, 880, 885, 890, 895, 900, 905, 910, 915, 920, 925, 1115, 1120, 1130])

In [2]:
img_folder = "E:\HD_VNese_map\main_src\samples"

In [5]:
import os
# convert extention of the images to jpg
for filename in os.listdir(img_folder):
    ext = filename.split(".")[-1]
    if ext != "jpg":
        os.rename(os.path.join(img_folder, filename), os.path.join(img_folder, filename.split(".")[0] + ".jpg"))

In [6]:
import cv2
import numpy as np
orange = np.uint8([[[0, 0, 255]]])
hsv_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2HSV)
print(hsv_orange)

[[[  0 255 255]]]


In [15]:
import cv2 
import numpy as np

def orange_color_identifier(img, threshold=10):
    """
    This function takes an image as input and returns True if the image contains a significant amount of orange color,
    and False otherwise. The threshold parameter is used to determine the minimum percentage of orange pixels required
    to return True. The default threshold is 10%.
    (adjust the lower and upper bounds for the orange color to match the specific shade of orange in the image.)
    """


    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the orange color
    lower_orange = np.array([0, 30, 30])
    upper_orange = np.array([30, 255, 255])

    # Create a mask that selects only the orange pixels
    mask = cv2.inRange(hsv_image, lower_orange, upper_orange)
    cv2.imshow("mask", mask)
    cv2.waitKey(0)
    # Count the number of orange pixels
    orange_pixels = np.sum(mask > 0)

    # Calculate the percentage of orange pixels
    total_pixels = img.shape[0] * img.shape[1]
    percentage_orange = (orange_pixels / total_pixels) * 100
    #print("percentage_orange", percentage_orange)   
    print("percentage_orange", percentage_orange)
    if percentage_orange < threshold:
        return False
    else:
        return True


In [16]:
import matplotlib.pyplot as plt
import os
img_folder = "E:\HD_VNese_map\main_src\samples"
# Load the image
for filename in os.listdir(img_folder):
    img = cv2.imread(os.path.join(img_folder, filename))
   
    if img is None:
        continue
    if orange_color_identifier(img, 5):
        print(filename, "contains orange color")
    else:
        print(filename, "does not contain orange color")
    
plt.show()

percentage_orange 34.25
1.jpg contains orange color
percentage_orange 63.51656249999999
10.jpg contains orange color
percentage_orange 22.78648926902669
2.jpg contains orange color
percentage_orange 11.293073442394709
3.jpg contains orange color
percentage_orange 26.32636591719002
4.jpg contains orange color
percentage_orange 0.0
5.jpg does not contain orange color
percentage_orange 20.709239587136558
6.jpg contains orange color
percentage_orange 0.596875
8.jpg does not contain orange color
